In [22]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-211667
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-211667


In [23]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [24]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd

# Specify parameter sampler
parameter_space =     {
        '--C': uniform(0.001, 1.0), # Regularization
        '--max_iter': choice(5, 20, 100, 200) # Max number of iterations (aka epochs)
    }
ps = RandomParameterSampling(parameter_space)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.3)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

config = ScriptRunConfig(source_directory='.',
                        script='train.py',
                        # arguments=['--arg1', arg1_val, '--arg2', arg2_val],
                        compute_target=cpu_cluster,
                        environment=sklearn_env
                        )
 
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=config, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name="Accuracy", 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=10,
                          max_concurrent_runs=8)

In [25]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
 
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_f9faf044-b587-411c-8e63-ca039581e72a',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-10-16T17:14:02.697519Z',
 'endTimeUtc': '2022-10-16T17:19:38.712918Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '28915a26-7cfb-4f09-b574-5032548c7158',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9141122913505311',
  'best_child_run_id': 'HD_f9faf044-b587-411c-8e63-ca039581e72a_7',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f9faf044-b587-411c-8e63-ca039581e72a_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

In [ ]:
cpu_cluster.delete()

In [26]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization Rate:',parameter_values[1])
print('Number of iterations:',parameter_values[3])

# model = best_run.register_model(model_name='model_hd', model_path='outputs/model.joblib')

Best Run Id:  HD_f9faf044-b587-411c-8e63-ca039581e72a_7
Accuracy: 0.9141122913505311
Regularization Rate: 0.09530162738301497
Number of iterations: 200


In [27]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

# Create TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [28]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df_clean = pd.concat([x, y], axis=1)

In [29]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=df_clean,
    label_column_name="y",
    n_cross_validations=6)

In [30]:
# Submit your automl run
print('Submitting Auto ML experiment...')
automl_exp = Experiment(ws, 'automl')
automl_run = automl_exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...


2022-10-16:17:20:45,356 WARNING  [local_experiment_driver.py:145] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_546ef577-ccbe-4675-b8ad-806461d898e8,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-10-16:17:56:13,631 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_546ef577-ccbe-4675-b8ad-806461d898e8,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

2022-10-16:18:04:53,602 INFO     [_session.py:46] Failed to load or parse file /home/azureuser/.azureml/auth/azureProfile.json. It will be overridden by default settings.
2022-10-16:18:04:53,606 INFO     [_session.py:46] Failed to load or parse file /home/azureuser/.azureml/auth/az.json. It will be overridden by default settings.
2022-10-16:18:04:53,609 INFO     [_session.py:46] Failed to load or parse file /home/azureuser/.azureml/auth/az.sess. It will be overridden by default settings.
2022-10-16:18:04:53,729 INFO     [_profile.py:149] No web browser is available. Fall back to device code.
2022-10-16:18:04:53,852 WARNING  [identity.py:131] To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ES4A23N6P to authenticate.
